In [1]:
import numpy as np
import threading
import time

class SimAxis:
    """Simulate a positioning motor."""

    readback = 0.
    setpoint = 0.
    tolerance = 0.001
    noise_amplitude = 0.0002
    done = True
    low_limit = -1
    high_limit = 121
    units = "mm"
    _poll_interval_s = 0.005
    _max_move = 0.05
    _stopping = False
    _controller_thread = None

    def __init__(self, prefix, name=None):
        self.prefix = prefix
        if name is None:
            raise ValueError("Must specify keyword argument 'name' with a string value.")
        self.name = name

    def __repr__(self, *args, **kwargs):
        nm = self.__class__.__name__
        arglist = [f'"{self.prefix}"',]
        for attr in "name readback setpoint units".split():
            v = getattr(self,attr)
            if isinstance(v, str):
                arglist.append(f'{attr}="{v}"')
            else:
                arglist.append(f"{attr}={v}")
        return f"{nm}({', '.join(arglist)})"

    def __str__(self, *args, **kwargs):
        return f"{self.name} = {self.position:.3f} {self.units}"

    def check_limits(self, value):
        if value < self.low_limit:
            raise ValueError(f"{value} is less than low limit: {self.limits}")
        elif value > self.high_limit:
            raise ValueError(f"{value} is greater than high limit: {self.limits}")

    def _controller(self, *value, **kwargs):
        # print(f"{self.moving = }")
        while self.moving:
            if self._stopping:
                print(f"stopping ...  {self.readback = }")
                self._stopping = False
                self.setpoint = self.readback
                break
            if not self.done:
                self._step()
            time.sleep(self._poll_interval_s)
            # print(f"{self.moving = }")
        self.done = True
        self._controller_thread = None

    @property
    def limits(self):
        return (self.low_limit, self.high_limit)

    def move(self, value):
        self.check_limits(value)
        # print(f"move({value})")
        self.setpoint = float(value)
        self.done = False
        runner = threading.Thread(target=self._controller)
        runner.start()
        self._controller_thread = runner

    @property
    def moving(self):
        diff = float(self.readback) - float(self.setpoint)
        return abs(diff) > self.tolerance

    @property
    def position(self):
        return self.readback

    def stop(self):
        if not self.done:
            print("stop received")
            self._stopping = True

    def _step(self):
        """simulate a motion"""
        diff = self.setpoint - self.readback
        sign = {True: 1, False: -1}[diff >= 0]
        step = sign*min(abs(diff)/2, self._max_move)
        noise = self.noise_amplitude*np.random.normal()
        target = step+noise
        # print(f"{diff = }")
        # print(f"{step = }")
        # print(f"{noise = }")
        # print(f"{target = }")

        self.readback += max(min(target, self.high_limit), self.low_limit)
        # print(f"{self.readback = }")


def move(axis, value, wait=True):
    axis.move(value)
    if not wait:
        return 0
    t0 = time.time()
    while not axis.done:
        time.sleep(0.1)
    return time.time()-t0

In [2]:
x = SimAxis("", name="x")
print(f"{x!r}")

SimAxis("", name="x", readback=0.0, setpoint=0.0, units="mm")


In [3]:
print(move(x, 1), "s")
print(repr(x))

print(move(x, 10), "s")
print(repr(x))

0.4395763874053955 s
SimAxis("", name="x", readback=0.9992408659551012, setpoint=1.0, units="mm")
2.9539873600006104 s
SimAxis("", name="x", readback=9.99916192626203, setpoint=10.0, units="mm")


In [4]:
# test the limits
for v in (-12_345, 543_212):
    try:
        x.move(v)
    except ValueError as exc:
        print(exc)

-12345 is less than low limit: (-1, 121)
543212 is greater than high limit: (-1, 121)


In [5]:
# relative move, don't wait
move(x, x.position+10, wait=False)

# let it run a short time
time.sleep(1.5)

# stop the move, report how long it takes to stop
t0 = time.time()
x.stop()
while not x.done:
    time.sleep(0.01)
print(f"time to stop: {time.time()-t0:.3f}s  {x!r}")

stop received
stopping ...  self.readback = 14.948498655521565
time to stop: 0.031s  SimAxis("", name="x", readback=14.948498655521565, setpoint=14.948498655521565, units="mm")


In [6]:
# a series of moves
for v in range(10, 1, -1):
    print(f"{move(x, v):.3f}s  {x!r}")

1.634s  SimAxis("", name="x", readback=10.000756700278446, setpoint=10.0, units="mm")
0.433s  SimAxis("", name="x", readback=9.000910179636273, setpoint=9.0, units="mm")
0.436s  SimAxis("", name="x", readback=8.000859645394701, setpoint=8.0, units="mm")
0.436s  SimAxis("", name="x", readback=7.000677039719834, setpoint=7.0, units="mm")
0.434s  SimAxis("", name="x", readback=6.000622911141888, setpoint=6.0, units="mm")
0.434s  SimAxis("", name="x", readback=5.000472499777437, setpoint=5.0, units="mm")
0.436s  SimAxis("", name="x", readback=4.000750864300072, setpoint=4.0, units="mm")
0.436s  SimAxis("", name="x", readback=3.0009647117165374, setpoint=3.0, units="mm")
0.435s  SimAxis("", name="x", readback=2.000742085239298, setpoint=2.0, units="mm")


In [7]:
# measure the movement speed for short and long moves
print(repr(x))
for hi in (-0.1, 1, 2, 5, 10, 20, 30):
    for v in (hi, 0):
        v0 = x.position
        dt = move(x, v)
        dx = x.position - v0
        s = dx/dt
        print(f"{dt:.3f}s  {dx = :.2f}{x.units}  {s = :.3f}{x.units}/s")

SimAxis("", name="x", readback=2.000742085239298, setpoint=2.0, units="mm")
0.754s  dx = -2.10mm  s = -2.784mm/s
0.106s  dx = 0.10mm  s = 0.925mm/s
0.436s  dx = 1.00mm  s = 2.295mm/s
0.440s  dx = -1.00mm  s = -2.271mm/s
0.758s  dx = 2.00mm  s = 2.636mm/s
0.762s  dx = -2.00mm  s = -2.625mm/s
1.634s  dx = 5.00mm  s = 3.059mm/s
1.739s  dx = -5.00mm  s = -2.875mm/s
3.262s  dx = 10.00mm  s = 3.065mm/s
3.265s  dx = -10.00mm  s = -3.063mm/s
6.303s  dx = 20.00mm  s = 3.173mm/s
6.315s  dx = -20.00mm  s = -3.167mm/s
9.449s  dx = 30.00mm  s = 3.175mm/s
9.443s  dx = -30.00mm  s = -3.177mm/s
